In [3]:
# Avoids reloading kernel while developing
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from PyCorrelationMatrixManager.correlation_matrix import *

import os

from src.utilities import *
from src.print_utilities import *

#pretty latex printing
from IPython.display import display, Math 
pprint = lambda o : display(Math(str(o)))



## Operators

Operators for SU(4) baryons are constructed in full at [su4ops](github.com/chrisculver/su4ops) or a repository of Kimmy Cushman.  Here they are translated into the correct objects for the WickContraction library.  Specifically at the moment the below code sets up a list of "Elemental Operators" in different irreps, with a gamma matrix name and spatial coordinate.  

In [5]:
# load definition of the building blocks of the operators.
from src.udud_elementals import *
import numpy as np

#gammas = [('buchoff',0)]
gammas=[]
gammas += np.array([[('A1g',i) for i in range(1)] for row in range(1)]).reshape(1,2).tolist()
gammas += np.array([[('T1g{}'.format(row),i) for i in range(1)] for row in range(1)]).reshape(1,2).tolist()
gammas += np.array([[('Eg{}'.format(row),i) for i in range(2)] for row in range(2)]).reshape(4,2).tolist()

print(gammas)

gammaName = lambda irrep,i : "\\Gamma^{{ ({},{}) }}".format(irrep, i)

aOps=[] # annihilation operators
cOps=[] # creation operators
for g in gammas:
    aOps.append(Operator([baryonSink(1,['u','u','u','u'],'[000]',gammaName(g[0],g[1]))]))
    cOps.append(Operator([baryonSource(1,['u','u','u','u'], '[000]',gammaName(g[0],g[1]))]))
    
    #aOps.append(Operator([baryonSink([1,-1,+1,0.5],[['d','u','u','u'],['u','d','u','u'],...],'x_0',gammaName(g[0],g[1]))]))

[['A1g', '0'], ['T1g0', '0'], ['Eg0', '0'], ['Eg0', '1'], ['Eg1', '0'], ['Eg1', '1']]


In [6]:
pprint(aOps[0])

<IPython.core.display.Math object>

In [7]:
pprint(cOps[0])

<IPython.core.display.Math object>

## Correlation Matrix Manager

The PyCorrelationMatrixManager does all the hard work for us.  We just need to give it the following information.

* List of creation operators
* List of annihilation operators
* List of gamma matrices
* List of dts to compute correlators on
* List of t0s - source times to average over
* Configuration number
* List of files containing numeric data for the diagrams

The output are correlators for all dts for the given configuration number.

In [15]:
diagramFile=os.path.join(os.path.expanduser("~"),"Code","LQCD","su4workflow","Input","diagrams_")
corrMatrix = CorrelationMatrix(
    cops=cOps[0:2], aops=aOps[0:2], gammas=gammas, 
    dts=[t for t in range(32)], t0s=[t for t in range(32)], 
    cfg=335, dfiles=[diagramFile])
corrMatrix.run()

Now we rerun the code without the diagramFiles, since it doesn't have numerical data to load, it gives us the necessary files to run the CPP code.

In [16]:
corrMatrix = CorrelationMatrix(
    cops=cOps[0:2], aops=aOps[0:2], gammas=gammas, 
    dts=[t for t in range(32)], t0s=[t for t in range(32)], 
    cfg=335)
corrMatrix.run()

TODO: This can be massively improved
      For now just tells you the full calculation
TODO: Need to handle momenta and baryon flavor!


RuntimeError: Failed to load all diagram values